In [1]:
import os

In [2]:
%pwd

'/home/kalema/Projects/ML/Wine-Quality-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/kalema/Projects/ML/Wine-Quality-MLflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kalema3502/Wine-Quality-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kalema3502"
os.environ["MLFLOW_TRACKING_PASSWORD"]="fb3845efcc3b2e46a4157b1d2c977a21e02dd16e"

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [9]:
from wineQuality.constants import *
from wineQuality.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            target_column=schema.name,
            all_params=params,
            metric_file_name=config.metric_file_name,
            mlflow_uri="https://dagshub.com/kalema3502/Wine-Quality-MLflow.mlflow"
        )

        return model_evaluation_config

In [12]:
import pandas as pd
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
import numpy as np

In [13]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config
    
    def eval_metrics(self, actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)

        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted = model.predict(test_x)

            rmse, mae, r2 = self.eval_metrics(test_y, predicted)

            scores = {"rmse": rmse, "mae": mae, "r2_score": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2_score", r2)

            signature = infer_signature(test_x, predicted)

            if tracking_uri_type_store == "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel", signature=signature)
            else:
                mlflow.sklearn.log_model(model, "model")

In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-30 10:06:10,742: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-30 10:06:10,761: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-30 10:06:10,797: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-30 10:06:10,809: INFO: common: Created directory at: artifacts/model_evaluation]
[2024-03-30 10:06:12,962: INFO: common: Json file saved at: artifacts/model_evaluation/metrics.json]


/home/kalema/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
